# Logic trees for Nath & Thingbaijam (2012)

Read logic tree tables extracted from publication, along with a table of supplementary information write them to TEX and XML. 

Notes:
  1. The trees require information from the source models, so if changes are made there then this must be rerun too.
  2. Symbolic links to the source models must exist in this folder
  3. For imports to work, ../utilities directory must be added to PYTHONPATH

_Copyright (C) 2016-2018 Nick Ackerley_

_This program is free software: you can redistribute it and/or modify
it under the terms of the GNU Affero General Public License as
published by the Free Software Foundation, either version 3 of the
License, or (at your option) any later version._

_This program is distributed in the hope that it will be useful,
but WITHOUT ANY WARRANTY; without even the implied warranty of
MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE. See the
GNU Affero General Public License for more details._

_You should have received a copy of the GNU Affero General Public 
License along with this program. If not, see 
<https://www.gnu.org/licenses/>._

In [1]:
import os
import sys
from shutil import copyfile
from IPython.display import display

import pandas as pd

from openquake.hazardlib import nrml
from openquake.qa_tests_data import classical

from logic_tree_tools import (
    nrml_to_pdf, read_tree_tsv, write_gsim_tree_nrml, df_to_tree,
    expand_sources, strip_fqtag, get_dict_key_match)

/home/nackerle/anaconda3/envs/oq/lib/python3.5/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
MAX_VERSION = 2
GSIM_TREE_TSV_LIST = ['gmpe_logic_tree_v%d.tsv' % version for version in range(MAX_VERSION + 1)]
SOURCE_TREE_TSV_LIST = [file % version for file in [
    'source_model_logic_tree_v%d.tsv',
    'areal_model_logic_tree_v%d.tsv',
    'collapsed_v%d.tsv',
] for version in range(MAX_VERSION + 1)]

## GMPE Logic Tree

In [4]:
for gsim_tree_tsv in GSIM_TREE_TSV_LIST:
    gsim_tree_df = read_tree_tsv(gsim_tree_tsv)
    gsim_tree_df.drop(['Tectonic Region Type', 'Qualifier', 'Short Names'],
                      axis=1)
    gsim_tree = df_to_tree(gsim_tree_df, validate=False)
    
    gsim_tree_xml = gsim_tree_tsv.replace('.tsv','.xml')
    with open(gsim_tree_xml, 'wb+') as f:
        nrml.write([gsim_tree], f, fmt='%g')
    try:
        nrml_to_pdf(gsim_tree_xml, include_ids=False)
    except FileNotFoundError as ex:
        pass

Executing:lualatex -output-directory=build -interaction=nonstopmode gmpe_logic_tree_v0.tex
Executing:lualatex -output-directory=build -interaction=nonstopmode gmpe_logic_tree_v1.tex
Executing:lualatex -output-directory=build -interaction=nonstopmode gmpe_logic_tree_v2.tex


## Source Model Logic Tree

In [5]:
nrml_list = []
for source_tree_tsv in SOURCE_TREE_TSV_LIST:
    print("Converting to NRML: " + source_tree_tsv)
    source_tree_symbolic_df = read_tree_tsv(source_tree_tsv)
    display(source_tree_symbolic_df)
    
    source_tree_symbolic = df_to_tree(source_tree_symbolic_df, 
                                      validate=True)
    source_tree_symbolic_xml = source_tree_tsv.replace('.tsv',
                                                       '_symbolic.xml')
    with open(source_tree_symbolic_xml, 'wb+') as f:
        nrml.write([source_tree_symbolic], f, fmt='%g')
    nrml_list.append(source_tree_symbolic_xml)
        
    source_tree_df = expand_sources(source_tree_symbolic_df)
    display(pd.concat((source_tree_df.head(), source_tree_df.tail())))
    source_tree = df_to_tree(source_tree_df, validate=False)
    source_tree_xml = source_tree_tsv.replace('.tsv','.xml')
    with open(source_tree_xml, 'wb+') as f:
        nrml.write([source_tree], f, fmt='%g')
    nrml_list.append(source_tree_xml)
    print('')
    
for nrml_file in nrml_list:
    print("Converting to PDF: " + nrml_file)
    try:
        nrml_to_pdf(nrml_file, include_ids=False)
    except FileNotFoundError as ex:
        print(repr(ex))

Converting to NRML: source_model_logic_tree_v0.tsv


,uncertaintyType,applyToSources,uncertaintyModel,uncertaintyWeight
0,sourceModel,all,"[nt2012_areal_source_model_v0.xml, nt2012_smoo...","[0.4, 0.27, 0.33]"
1,maxMagGRAbsolute,nt2012_areal_source_model_v0.csv,"[mmax - stdmmax, mmax, mmax + stdmmax]","[0.32, 0.36, 0.32]"
2,bGRRelative,nt2012_areal_source_model_v0.csv,"[-stdb, 0, +stdb]","[0.32, 0.36, 0.32]"


Reading: /home/nackerle/src/indian-subcontinent-psha/Logic Trees/nt2012_areal_source_model_v0.csv
Reading: /home/nackerle/src/indian-subcontinent-psha/Logic Trees/nt2012_areal_source_model_v0.csv


,applyToSources,uncertaintyModel,uncertaintyType,uncertaintyWeight
0,all,"[nt2012_areal_source_model_v0.xml, nt2012_smoo...",sourceModel,"[0.4, 0.27, 0.33]"
1,z1m,"[7.6, 8.0, 8.4]",maxMagGRAbsolute,"[0.32, 0.36, 0.32]"
2,z2m,"[7.6, 8.0, 8.4]",maxMagGRAbsolute,"[0.32, 0.36, 0.32]"
3,z3,"[7.6, 8.0, 8.4]",maxMagGRAbsolute,"[0.32, 0.36, 0.32]"
4,z4,"[7.0, 7.4, 7.8]",maxMagGRAbsolute,"[0.32, 0.36, 0.32]"
218,z931m,"[-0.12, 0.0, 0.12]",bGRRelative,"[0.32, 0.36, 0.32]"
219,z932,"[-0.1, 0.0, 0.1]",bGRRelative,"[0.32, 0.36, 0.32]"
220,z933,"[-0.13, 0.0, 0.13]",bGRRelative,"[0.32, 0.36, 0.32]"
221,z935,"[-0.09, 0.0, 0.09]",bGRRelative,"[0.32, 0.36, 0.32]"
222,z936,"[-0.11, 0.0, 0.11]",bGRRelative,"[0.32, 0.36, 0.32]"



Converting to NRML: source_model_logic_tree_v1.tsv


...

,applyToSources,uncertaintyModel,uncertaintyType,uncertaintyWeight
0,all,"[nt2012_areal_source_model_v1_collapsed.xml, n...",sourceModel,"[0.4, 0.27, 0.33]"
0,all,"[nt2012_areal_source_model_v1_collapsed.xml, n...",sourceModel,"[0.4, 0.27, 0.33]"



Converting to NRML: collapsed_v2.tsv


,uncertaintyType,applyToSources,uncertaintyModel,uncertaintyWeight
0,sourceModel,all,"[nt2012_areal_source_model_v2_collapsed.xml, n...","[0.4, 0.27, 0.33]"


,applyToSources,uncertaintyModel,uncertaintyType,uncertaintyWeight
0,all,"[nt2012_areal_source_model_v2_collapsed.xml, n...",sourceModel,"[0.4, 0.27, 0.33]"
0,all,"[nt2012_areal_source_model_v2_collapsed.xml, n...",sourceModel,"[0.4, 0.27, 0.33]"



Converting to PDF: source_model_logic_tree_v0_symbolic.xml
Executing:lualatex -output-directory=build -interaction=nonstopmode source_model_logic_tree_v0_symbolic.tex
FileNotFoundError(2, 'No such file or directory')
Converting to PDF: source_model_logic_tree_v0.xml
Too many (223) nodes in logicTree, abbreviating TEX to first & last 5
Executing:lualatex -output-directory=build -interaction=nonstopmode source_model_logic_tree_v0.tex
FileNotFoundError(2, 'No such file or directory')
Converting to PDF: source_model_logic_tree_v1_symbolic.xml
Executing:lualatex -output-directory=build -interaction=nonstopmode source_model_logic_tree_v1_symbolic.tex
FileNotFoundError(2, 'No such file or directory')
Converting to PDF: source_model_logic_tree_v1.xml
Executing:lualatex -output-directory=build -interaction=nonstopmode source_model_logic_tree_v1.tex
FileNotFoundError(2, 'No such file or directory')
Converting to PDF: source_model_logic_tree_v2_symbolic.xml
Executing:lualatex -output-directory=b

In [6]:
node = source_tree_symbolic[0][0]
print(node)
print('%s = %dx %s' % (strip_fqtag(node.tag), len(node),
                       strip_fqtag(node[0].tag)))
print(get_dict_key_match(node.attrib, 'id'))
print(node[0].uncertaintyModel)

<logicTreeBranchSet {'branchSetID': 'bs1', 'uncertaintyType': 'sourceModel'} None ...>
logicTreeBranchSet = 3x logicTreeBranch
branchSetID
<uncertaintyModel {} nt2012_areal_source_model_v2_collapsed.xml >


In [7]:
node = source_tree[0][0]
print(node)
print('%s = %dx %s' % (strip_fqtag(node.tag), len(node),
                       strip_fqtag(node[0].tag)))
print(get_dict_key_match(node.attrib, 'id'))
model = node[0].uncertaintyModel
print(model)

<logicTreeBranchSet {'branchSetID': 'bs1', 'uncertaintyType': 'sourceModel'} None ...>
logicTreeBranchSet = 3x logicTreeBranch
branchSetID
<uncertaintyModel {} nt2012_areal_source_model_v2_collapsed.xml >
